In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from os import listdir, getenv
import sqlalchemy as sa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Labelling all flights with the model created

In [2]:
# Import the querying module
from flight_querying import query_flights
import pandas as pd

# Set up and retrieve the data from the database.
db_connect = query_flights()

In [3]:
# import model to label new data
import joblib
model_filename = 'ML_model_outputs/label_xgboost_model.joblib'
model = joblib.load(model_filename)

In [12]:
# query the list of flight ids excluding these six which are already labelled
flight_ids = db_connect.get_flights("Flight test", ["id"], "flights")
flight_ids = flight_ids['id'].to_list()
manually_labelled_ids = [4620, 4929, 4940, 5019, 5021, 5034, 4636, 4842, 4868, 4925, 4978, 5362, 5116]

# remove the manually_labelled_ids from the list 
flight_ids = [id for id in flight_ids if id not in manually_labelled_ids]

In [13]:
flight_ids

[5367,
 5205,
 5127,
 5117,
 5109,
 5064,
 5071,
 5074,
 5072,
 5037,
 5039,
 5190,
 5192,
 5194,
 5096,
 5023,
 5025,
 5013,
 4999,
 4994,
 4990,
 4987,
 4992,
 4983,
 4981,
 4976,
 4975,
 4938,
 4936,
 4931,
 4927,
 4923,
 4921,
 4919,
 4917,
 4910,
 4915,
 4909,
 4906,
 4908,
 4904,
 4903,
 4901,
 4871,
 4862,
 4860,
 4857,
 4853,
 4850,
 4845,
 4802,
 4795,
 4792,
 4790,
 4785,
 4783,
 4780,
 4766,
 4669,
 4633,
 4622]

In [14]:
# Fetch data for specified flight IDs # first id in list is 5367
data_frames = [db_connect.get_flightdata_for_ml_data_label(flight_id) for flight_id in flight_ids]

# Concatenate data frames
x = pd.concat(data_frames, axis=0).dropna()

In [15]:
# drop id column for model prediction
id_column = x[['id']]
x = x.drop(columns=["id"])

In [16]:
# Make predictions on the test set
predictions = model.predict(x)

In [17]:
# insert the predicted values back into the x dataframe
x['activity'] = predictions
x['flight_id'] = id_column

In [18]:
# replace values in activity column with their string activity mapping
# labels = ['NA', 'climb', 'cruise', 'descent', 'landing', 'power off stall',
#           'power on stall', 'slow flight', 'steep turns', 'takeoff']

labels = ['HASEL', 'NA', 'climb', 'cruise', 'descent', 'landing', 'post-flight',
          'power off stall', 'power on stall', 'pre-flight', 'slow flight',
          'steep turn', 'steep turns', 'takeoff']

x['activity'] = x['activity'].map(lambda x: labels[x])

In [19]:
def connect_to_db(provider: str):
    load_dotenv()
    provider == "PostgreSQL"
    db_url = "postgresql+psycopg2" + getenv('DATABASE_URL')[8:]
    engine = sa.create_engine(db_url, connect_args={"options": "-c timezone=US/Eastern"})
    return engine

In [20]:
# trim all columns except for the ones in flight_activities table
flight_activities_data = x[['flight_id', 'time', 'activity']]
flight_activities_data = flight_activities_data.rename(columns={"time": "time_min"})
flight_activities_data.head()

# push the updated data to the flight_activities table
# engine = db_connect.connect()
engine = connect_to_db("PostgreSQL")
flight_activities_data.to_sql('flight_activities', engine, if_exists='append', index=False)
engine.dispose()